In [1]:
import os
from scipy.sparse.csgraph import connected_components
import umap
from lightgbm import early_stopping
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline



c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\user\AppData\Local\Programs\Python\Python310\

In [ ]:

train = pd.read_csv(
    'train_df.csv')
test = pd.read_csv('test_df.csv')

train['body_system_2'] = train['body_system_2'].replace(
    'Undefined Diagnoses', 'Undefined diagnoses')
test['body_system_2'] = test['body_system_2'].replace(
    'Undefined Diagnoses', 'Undefined diagnoses')
